## Setup

In [ ]:
import pandas as pd
import sqlite3 as s3
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn import metrics
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')

go_sales = s3.connect('go_sales.sqlite')

order_details = pd.read_sql_query('SELECT * FROM order_details', go_sales)
order_header = pd.read_sql_query('SELECT * FROM order_header', go_sales)
returned_item = pd.read_sql_query('SELECT * FROM returned_item', go_sales)

## Trainset

In [ ]:
merged_orders = pd.merge(order_details, order_header, left_on='ORDER_NUMBER', how='inner', right_on='ORDER_NUMBER')
df = pd.merge(merged_orders, returned_item, left_on='ORDER_DETAIL_CODE', how='outer', right_on='ORDER_DETAIL_CODE')
df = df[['PRODUCT_NUMBER', 'QUANTITY', 'UNIT_PRICE', 'UNIT_SALE_PRICE', 'RETAILER_SITE_CODE', 'ORDER_DATE', 'RETURN_DATE', 'RETURN_REASON_CODE']]

# Om de RETURN_DATE mee te geven aan de trainset. Format de data en vervolgens afsplitsen in dag maand jaar.
df['RETURN_DATE'] = pd.to_datetime(df['RETURN_DATE'], format="%d-%m-%Y %H:%M:%S")
df['RETURN_DAY'] = df['RETURN_DATE'].dt.day
df['RETURN_MONTH'] = df['RETURN_DATE'].dt.month
df['RETURN_YEAR'] = df['RETURN_DATE'].dt.year
df = df.drop('RETURN_DATE', axis=1)

# Om de afgesplitste RETURN_DATE dagen maanden en jaren te vullen met 0, als de bestelling niet is geretourneerd en dus geen RETURN_DATE heeft.
df['RETURN_DAY'] = df['RETURN_DAY'].fillna(0).astype(int)
df['RETURN_MONTH'] = df['RETURN_MONTH'].fillna(0).astype(int)
df['RETURN_YEAR'] = df['RETURN_YEAR'].fillna(0).astype(int)

# Om de ORDER_DATE af te splitsen in dag maand en jaar.
df['ORDER_DATE'] = pd.to_datetime(df['ORDER_DATE'])
df['ORDER_DAY'] = df['ORDER_DATE'].dt.day
df['ORDER_MONTH'] = df['ORDER_DATE'].dt.month
df['ORDER_YEAR'] = df['ORDER_DATE'].dt.year
df = df.drop('ORDER_DATE', axis=1)

# Vult de RETURN_REASON_CODE met 0 voor de bestellingen die niet geretourneerd zijn.
df['RETURN_REASON_CODE'] = df['RETURN_REASON_CODE'].fillna(0).astype(int)

# Een filter om de trainset alleen de geretourneerde bestellingen mee te geven. Comment voor álle bestellingen, uncomment voor alleen de geretourneerde bestellingen.
df = df[df['RETURN_REASON_CODE'] != 0]
df

In [ ]:
x = df.drop('RETURN_REASON_CODE', axis = 1)
y = df.loc[:, ['RETURN_REASON_CODE']]

x_train, x_text, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=35)

dtree = DecisionTreeClassifier(max_depth=2)
dtree = dtree.fit(x_train, y_train)
tree.plot_tree(dtree, feature_names=x.columns)
plt.show()

In [ ]:
predicted_df = pd.DataFrame(dtree.predict(x_text))
predicted_df = predicted_df.rename(columns={0:'Predicted Reason Code'})
model_results_frame = pd.concat([y_test.reset_index()['RETURN_REASON_CODE'], predicted_df], axis=1)
model_results_frame

In [ ]:
confusion_matrix = metrics.confusion_matrix(model_results_frame['RETURN_REASON_CODE'], model_results_frame['Predicted Reason Code'])

# Uncomment voor álle bestellingen.
# cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [0, 1, 2, 3, 4, 5])
# Uncomment voor alleen de geretourneerde bestellingen.
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [1, 2, 3, 4, 5])

cm_display.plot()
plt.show()

In [ ]:
metrics.accuracy_score(model_results_frame['RETURN_REASON_CODE'], model_results_frame['Predicted Reason Code'])